In [1]:
import tensorflow as tf

In [2]:
tf.executing_eagerly() 

False

In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Bidirectional
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import os
import nltk
import pymagnitude

In [6]:
data = pd.read_json("data.json")
data['comments'] = [" ".join(x) for x in data.comments]

validLetters = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789 "
def removeALLPunctuation(text=""): 
   t = (text.lower().replace("."," ").replace(","," ").replace(":"," ").replace(")"," ")
        .replace("("," ").replace("-"," ").strip())
   re.sub( '\s+', ' ', t ).strip()
   return ''.join([char for char in t if char in validLetters]).strip()

data.comments = [removeALLPunctuation(c) for c in data.comments] 
data.title = [removeALLPunctuation(c) for c in data.title] 
data.comments = [nltk.word_tokenize(c) for c in data.comments]
data.title = [nltk.word_tokenize(c) for c in data.title]

In [7]:
data.head()

,comments,title,url
0,"[i, dont, even, know, what, to, say, rtoastme,...","[i, wanted, to, share, a, big, milestone, for,...",https://i.redd.it/y2u2a7gxuuj21.jpg
1,"[dude, i, like, what, i, see, on, my, screen, ...","[having, a, bad, mental, health, day, not, lik...",https://i.redd.it/9ir4yomepan21.jpg
2,"[youre, beautiful, really, very, beautiful, yo...","[lost, my, hair, and, my, breasts, to, cancer,...",https://i.redd.it/roomyzsyw0e21.jpg
3,"[you, look, like, the, coolest, chillest, guy,...","[16m, still, having, issues, with, self, worth...",https://i.redd.it/oxqwcainofp21.jpg
4,"[good, on, you, you, will, inspire, everyone, ...","[in, the, last, 5, months, i, worked, up, the,...",https://i.redd.it/4n8ceujdwhm21.jpg


In [8]:
max_fatures = 2000
max_length = 62
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data.comments.values + data.title.values)
X = tokenizer.texts_to_sequences(data.title.values)
X = pad_sequences(X)
y = tokenizer.texts_to_sequences(data.comments.values)
y = pad_sequences(y, maxlen=max_length)

In [9]:
X.shape, y.shape

((993, 62), (993, 62))

# PREP EMBEDDING LAYER

In [10]:
GLOVE_DIR = "../glove/"
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.300d.txt'), encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [11]:
EMBEDDING_DIM = 300
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, EMBEDDING_DIM))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# Compile Model

In [12]:
from keras.layers import TimeDistributed
from keras.layers import Input
from keras.models import Model

In [13]:
x_length = X.shape[1]
y_length = y.shape[1]

In [14]:
y.shape

(993, 62)

In [15]:
len(tokenizer.word_index)

26356

In [16]:
def one_hot_encode(a):
    return (np.arange(len(tokenizer.word_index)) == a[...,None]-1).astype(int)

In [17]:
vocab_size = len(tokenizer.word_index)

In [18]:
y_one_hot = one_hot_encode(y)

In [19]:
tokenizer.sequences_to_texts([[np.argmax(w) for w in y_one_hot[1]]])

['laugh yet for keep put also everywhere i love reason here sorry a good the after you is pretty edit love what time is to school too she good and ever to each in love you has so yet is to smile 4 up its you young coming own that about a proud a nice been brave one']

In [20]:
y_one_hot.shape

(993, 62, 26356)

In [21]:
latent_dim = 3
decoder_input_data = y
decoder_target_data = y_one_hot

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(len(tokenizer.word_index) + 1,
                    EMBEDDING_DIM,
                    weights=[embedding_matrix],
                    input_length=x_length,
                    trainable=False)(encoder_inputs)
encoder = LSTM(latent_dim, return_state=True)
encoder_lstm, state_h, state_c = encoder(encoder_embedding)
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
decoder_embedder = Embedding(len(tokenizer.word_index) + 1,
                    EMBEDDING_DIM,
                    weights=[embedding_matrix],
                    input_length=y_length,
                    trainable=False)
decoder_embedding = decoder_embedder(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True)
decoder_outputs = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile & run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
# Note that `decoder_target_data` needs to be one-hot encoded,
# rather than sequences of integers like `decoder_input_data`!
print(model.summary())
model.fit([X, decoder_input_data], decoder_target_data,
          batch_size=32,
          epochs=1)

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 62, 300)      7907100     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 62, 300)      7907100     input_2[0][0]                    
_____________________________________

In [30]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_embedding = decoder_embedder(decoder_inputs)
#decoder_outputs, state_h, state_c = tf.map_fn(lambda x: decoder_lstm(x, initial_state=decoder_states_inputs), decoder_embedding)
decoder_outputs = decoder_lstm(decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

ValueError: Layer lstm_2 expects 5 inputs, but it received 3 input tensors. Input received: [<tf.Tensor 'embedding_2_5/embedding_lookup/Identity:0' shape=(?, ?, 300) dtype=float32>, <tf.Tensor 'input_11:0' shape=(?, 3) dtype=float32>, <tf.Tensor 'input_12:0' shape=(?, 3) dtype=float32>]

In [29]:
decode_sequence("hello")

AttributeError: 'str' object has no attribute 'ndim'